In [1]:
from pynq_dpu import DpuOverlay
import numpy as np
import cv2
from IPython.display import Image
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import os

In [2]:
overlay = DpuOverlay("dpu.bit")
overlay.load_model("adam_100_da_best_quantized.xmodel")

In [3]:
# Load label mapping
df = pd.read_csv('cards.csv')
label_encoder = LabelEncoder()
df["encoded_labels"] = label_encoder.fit_transform(df["labels"])
label_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
class_names = list(label_mapping.keys())

In [4]:
# Path to images
image_dir = "./test_images"  # directory with 10 identical images
image_paths = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".jpg")])

In [5]:
# DPU runner and tensor setup
dpu = overlay.runner
input_tensor = dpu.get_input_tensors()
output_tensor = dpu.get_output_tensors()
fix_point = dpu.get_input_tensors()[0].get_attr("fix_point")
scale = 2 ** fix_point
inputDim = tuple(dpu.get_input_tensors()[0].dims)
inputData = [np.empty(inputDim, dtype=np.int8)]

outputDim = tuple(dpu.get_output_tensors()[0].dims)
outputData = [np.empty(outputDim, dtype=np.int8)]


In [6]:
%%time
correct_predictions = 0
total_predictions = 0

for i, path in enumerate(image_paths):
    # Get the true label (from the directory structure or CSV file)
    split_path = path.split('/')
    split_path[-1] = split_path[-1].replace('.jpg', '')
    true_label = label_mapping.get(split_path[-1], None)

    if true_label is None:
        print(f"Warning: Label for image {path} not found!")
        continue
    
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (128, 128)) / 255.0
    image = (image * scale).round().astype(np.int8)
    image = image.astype(np.int8)
    
    inputData[0][0] = image
    
    job_id = dpu.execute_async(inputData, outputData)
    dpu.wait(job_id)

    # Get prediction
    class_idx = outputData[0][0].argmax()
    predicted_label = class_names[class_idx]

    print(f"[{i}] True label: {split_path[-1]} | Predicted: {predicted_label}")

    # Accuracy calculation
    if true_label == class_idx:
        correct_predictions += 1

    total_predictions += 1

accuracy = (correct_predictions / total_predictions) * 100 if total_predictions > 0 else 0
print(f"Accuracy: {accuracy:.2f}% ({correct_predictions}/{total_predictions})")

[0] True label: four of hearts | Predicted: four of hearts
[1] True label: jack of clubs | Predicted: jack of clubs
[2] True label: king of diamonds | Predicted: king of diamonds
[3] True label: king of spades | Predicted: king of spades
[4] True label: nine of diamonds | Predicted: eight of hearts
[5] True label: nine of spades | Predicted: nine of spades
[6] True label: six of diamonds | Predicted: six of diamonds
[7] True label: six of hearts | Predicted: six of hearts
[8] True label: six of spades | Predicted: six of spades
[9] True label: ten of diamonds | Predicted: eight of diamonds
Accuracy: 80.00% (8/10)
CPU times: user 50 ms, sys: 3.78 ms, total: 53.7 ms
Wall time: 63.6 ms
